## Data Extraction and Manipulation from IMDB text files

<h2>Links</h2>

<ul>
  <li>
  <a href="https://docs.python.org/2/library/sqlite3.html">https://docs.python.org/2/library/sqlite3.html</a>
  </li>
  <li>
  <a href="http://www.sqlite.org/quickstart.html">http://www.sqlite.org/quickstart.html</a>
  </li>
  <li>
  <a href="http://imdbpy.sourceforge.net">http://imdbpy.sourceforge.net </a>
  </li>
</ul>

In [1]:
import pandas as pd
import sqlite3 #pip install sqlite3
import subprocess
#import urllib
import omdb
from xml.etree import ElementTree as ET
import datetime 

Create & connect to local SQlite db 

In [2]:
#connect to db
#note: creates empty sql db if it does not exist. 
con = sqlite3.connect('imdbpy/imdb.db')
c = con.cursor()

Populate database from imdb text file downloads (shell script)

In [ ]:
#warning will take a long tome to run (P.S I actualy ran this from the terminal)
#result = subprocess.run(
#    ['imdbpy2sql.py -d ~/Downloads -u sqlite:imdb.db --sqlite-transactions', '-l']
#    , stdout=subprocess.PIPE)
#result.stdout

DB Admin Tasks

In [3]:
#create index on movie type (kind_type) ie movie, tv ect
c.execute("CREATE INDEX IF NOT EXISTS title_idx_kind on title (kind_id)")

#rebuilds the database file, repacking it into a minimal amount of disk space
c.execute("VACUUM")

In [4]:
#list the tables
str_sql = "SELECT name as tables FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(str_sql, con)
tables

,tables
0,name
1,sqlite_sequence
2,char_name
3,company_name
4,kind_type
5,title
6,company_type
7,aka_name
8,aka_title
9,role_type


Roll up multiple genres per movie into a piped string

In [15]:
#Drop table first incase the script is run a second time
c.execute('DROP TABLE IF EXISTS movie_genre;')

#create table  
# movie_id    : Primary Key, unique identifer for movie
# genre       : Pipe seperated list of genres for the movie ie Romantic | Comedy
# genre_count : The number of genres ie 2 for Romantic | Comedy

#genres (info_type_id = 3)
#movie  (kind_id = 1)
str_sql = 'CREATE TABLE movie_genre as                      ' + \
          'SELECT                                           ' + \
          '   i.movie_id                   as movie_id      ' + \
          ' , group_concat(i.info," | ")   as genre         ' + \
          ' , count(i.info)                as genre_count   ' + \
          'FROM title t                                     ' + \
          '  INNER JOIN movie_info i on (t.id = i.movie_id) ' + \
          'WHERE t.kind_id = 1                              ' + \
          '  AND i.info_type_id = 3                         ' + \
          '  AND i.info != "Short"                          ' + \
          'GROUP BY i.movie_id                              '       
        
c.execute(str_sql)

#create index on movie type (kind_type) ie movie, tv ect
c.execute("CREATE UNIQUE INDEX IF NOT EXISTS pk_movie_genre_movie_id on movie_genre (movie_id)")

#show example table data
query_result = pd.read_sql_query("select * from movie_genre LIMIT 5", con)
query_result

,movie_id,genre,genre_count
0,2891047,Comedy,1
1,2891048,Comedy,1
2,2891050,Drama,1
3,2891051,Animation,1
4,2891052,Comedy,1


Denormalise the movies language(s)

In [58]:
c.execute('DROP TABLE IF EXISTS movie_language;')

str_sql = 'CREATE TABLE movie_language as                    ' + \
          'SELECT                                            ' + \
          '   i.movie_id                   as movie_id       ' + \
          ' , group_concat(i.info," | ")   as language       ' + \
          ' , count(i.info)                as language_count ' + \
          'FROM title t                                      ' + \
          '  INNER JOIN movie_info i on (t.id = i.movie_id)  ' + \
          'WHERE t.kind_id = 1                               ' + \
          '  AND i.info_type_id = 4                          ' + \
          'GROUP BY i.movie_id                               '
c.execute(str_sql)

#create index on movie type (kind_type) ie movie, tv ect
c.execute("CREATE UNIQUE INDEX IF NOT EXISTS pk_movie_language_movie_id on movie_language (movie_id)")

#show example table data
query_result = pd.read_sql_query("select * from movie_language LIMIT 5", con)
query_result

,movie_id,language,language_count
0,2891047,English,1
1,2891048,English,1
2,2891050,English,1
3,2891051,French,1
4,2891052,English,1


- Extract a list of movie names
- The IMDB text files do not contain the unique imdb_id code for the movies. 
- The db comes form the files. So ... the db does not have the imdb_id

We will need to extract the imdb by querying using omdb API. But first we need a list of movie titles and production years

In [43]:
#Extract id, title & year from the local db
#id is the unique id for the movie in the local db. it is not the imdb_id
#note kind_id = 1 = movies

str_sql = 'SELECT id                            ' + \
            ', trim(substr(title,2)) as title   ' + \
            ', ltrim(production_year,4) as year ' + \
          'FROM title                           ' + \
          'WHERE kind_id = 1                    ' + \
          ' AND ifnull(production_year,0) <> 0  ' + \
          ' AND title <> "#"                    ' + \
          ' AND imdb_id IS NULL;                '
            
movie_list = pd.read_sql_query(str_sql, con)
movie_list.head(6)

,id,title,year
0,2891058,1 Dad,2016
1,2891102,Alleman,2013
2,2891103,allthatissolidmeltsintoair,2016
3,2891104,am/pm,2018
4,2891111,armoire,2016


Query omdb to gain imdb_id code

In [44]:
success = 0
failure = 0
for index, row in movie_list.iterrows():
    id    = row['id']
    year  = row['year']
    title = row['title'].strip()
    
    #query omdb
    try:
        res = omdb.request(t=title, y=year, r='xml', type='movie')
        xml_content = res.content
        
        #Check to see if the movie was matched
        root = ET.fromstring(xml_content)
        if(root.get('response') == 'True'):
            
            #extract the imdb_id from the top match
            imdbID = root.findall('movie')[0].get('imdbID')

            #write imdb_id back to database
            str_sql = "UPDATE title SET imdb_id = '" + imdbID + "' WHERE id = " + str(id) 
            c.execute(str_sql)
            success = success + 1
        else:
            failure = failure + 1
        
    except:
        failure = failure + 1

    #every thousand records: print status update 
    if index % 1000 == 0: 
        #commit changes to the DB
        con.commit()
        
        #Print status update 
        print index, success, failure, datetime.datetime.now().time()
    con.commit()        

0 0 1 17:20:28.614747


In [4]:
#Outputs SQL query to csv / utf-8 encoding
def sql_to_file(str_sql, file_name):
    df = pd.read_sql_query(str_sql, con)
    df.to_csv(file_name, encoding='utf-8')
    return df

Export map between local db keys and imdb_id (include pipe seperated genre data)

In [62]:
str_sql = 'SELECT id                                             ' + \
            ', t.imdb_id                                         ' + \
            ', trim(substr(t.title,2)) as title                  ' + \
            ', ltrim(t.production_year,4) as year                ' + \
            ', g.genre                                           ' + \
            ', g.genre_count                                     ' + \
            ', l.language                                        ' + \
            ', l.language_count                                  ' + \
          'FROM title t                                          ' + \
          ' LEFT OUTER JOIN movie_genre g                        ' + \
          '   on (t.id = g.movie_id)                             ' + \
          ' LEFT OUTER JOIN movie_language l                     ' + \
          '   on (t.id = l.movie_id)                             ' + \
          'WHERE t.kind_id = 1                                   ' + \
          ' AND t.imdb_id IS NOT NULL ;                          '

df = sql_to_file(str_sql, 'imdb_id.csv')
print 'exported ', len(df), ' records'
df.head(5)

exported  44821  records


,id,imdb_id,title,year,genre,genre_count,language,language_count
0,2891050,tt0491587,1,2005,Drama | Short,2.0,English,1.0
1,2891051,tt0408060,1,2009,Animation | Short,2.0,French,1.0
2,2891052,tt0926084,1,2010,Comedy | Short,2.0,English,1.0
3,2891055,tt4856314,1 at the Apocalypse Box Office,2015,Comedy | Short,2.0,English,1.0
4,2891056,tt3597346,1 Beauty Nail Salon,2014,Drama | History | Short | War,4.0,English,1.0


Export Genre Combination Summary

In [37]:
str_sql = 'SELECT                                           ' + \
            '  ltrim(t.production_year,4) as year           ' + \
            ', g.genre                    as genre_pair     ' + \
            ', avg(g.genre_count)         as pair_len       ' + \
            ', COUNT(t.id)                as movie_count    ' + \
          'FROM movie_genre as g                            ' + \
          '  INNER JOIN title t on (g.movie_id = t.id)      ' + \
          'GROUP BY ltrim(t.production_year,4)              ' + \
          '        , g.genre                                '
            
df = sql_to_file(str_sql, 'GenreCombinationSummary.csv')
print 'exported ', len(df), ' records'
df.head(5)

exported  58291  records


,year,genre_pair,pair_len,movie_count
0,None,Action,1.0,1523
1,None,Action | Adventure,2.0,107
2,None,Action | Adventure | Animation,3.0,5
3,None,Action | Adventure | Animation | Biography | C...,7.0,1
4,None,Action | Adventure | Animation | Comedy,4.0,3


Export genre count (not unique a movie may be counted in two genres)

In [42]:
str_sql = 'SELECT                                           ' + \
          '  LTRIM(t.production_year,4)   as year           ' + \
          ', i.info                       as genre          ' + \
          ', count(i.info)                as movie_count    ' + \
          'FROM title t                                     ' + \
          '  INNER JOIN movie_info i on (t.id = i.movie_id) ' + \
          'WHERE t.kind_id = 1                              ' + \
          '  AND i.info_type_id = 3                         ' + \
          'GROUP BY LTRIM(t.production_year,4)              ' + \
          '       , i.info                                  '
df = sql_to_file(str_sql, 'GenreSummary.csv')
print 'exported ', len(df), ' records'
df.head(5)

exported  3088  records


,year,genre,movie_count
0,None,Action,4103
1,None,Adult,121
2,None,Adventure,1720
3,None,Animation,660
4,None,Biography,805


In [16]:



str_sql = 'SELECT                                          ' + \
          '  genre                       as genre          ' + \
          ', count(*)                    as movie_count    ' + \
          ', avg(genre_count)            as genre_count    ' +\
          'FROM movie_genre                                ' + \
          'GROUP BY genre                                  ' + \
          'ORDER BY 2 DESC'  
            
df = sql_to_file(str_sql, 'GenreSummaryNoYear.csv')
print 'exported ', len(df), ' records'
df.head(5)

exported  6506  records


,genre,movie_count,genre_count
0,Drama,168075,1.0
1,Documentary,121552,1.0
2,Comedy,111909,1.0
3,Animation,21270,1.0
4,Horror,19858,1.0


In [28]:
str_sql = 'select sum(movie_count) from batshit where movie_count < 150'  

str_sql = 'select count(*) from batshit where movie_count >150'  



df = pd.read_sql_query(str_sql, con)
df.head(6)

,count(*)
0,262
